# RedPanda Examples

In [1]:
import pandas, redpanda, sqlalchemy, sys
print "RedPanda version:", redpanda.__version__
print "Python version:", sys.version

RedPanda version: 0.3.1
Python version: 2.7.12 (default, Jun 29 2016, 08:57:23) 
[GCC 5.3.0]


## Opening a session

Open a `RedPanda` session to begin exploring data:

In [2]:
engine = redpanda.create_engine("sqlite://")
Session = redpanda.orm.sessionmaker(bind=engine)
session = Session()

Note that this is syntactic sugar for SQLAlchemy calls:

In [3]:
engine = sqlalchemy.create_engine("sqlite://")
sessionmaker = sqlalchemy.orm.sessionmaker(class_=redpanda.orm.Session,
                                           query_cls=redpanda.orm.Query)
session = sessionmaker(bind=engine)

## Creating An Example Model

Define your models using the declarative API. `RedPanda` defines one such model in the `example` module:

In [4]:
import sqlalchemy.ext.declarative
Base = sqlalchemy.ext.declarative.declarative_base()
class Widget(Base):
    id        = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    timestamp = sqlalchemy.Column(sqlalchemy.DateTime)
    name      = sqlalchemy.Column(sqlalchemy.String)
    kind      = sqlalchemy.Column(sqlalchemy.String)
    units     = sqlalchemy.Column(sqlalchemy.Integer)

    __read_sql__  = {"index_col": ["timestamp"], "parse_dates": ["timestamp"]}
    __tablename__ = "widgets"

Defining the class attribute `__read_sql__` will configure default arguments passed to `pandas.read_sql()`. 

In this case we are configuring the `widgets` table to use the `timestamp` column as an index and that we wish for `pandas` to parse this column as a `datetime` object.

In [5]:
from redpanda.example import create_widgets
create_widgets(session)

## Querying Data

Use the `frame()` method of a query to return the results as a `DataFrame`:

In [6]:
session.query(Widget).frame().head()

,id,name,kind,units
timestamp,,,,
2016-01-05 10:07:00,1,injuries,bopper,54
2016-01-08 08:23:00,2,trick,buzzer,55
2016-01-12 07:54:00,3,flashlights,bopper,93
2016-01-24 13:10:00,4,purposes,fizzer,48
2016-01-29 12:13:00,5,subsystems,bopper,34


### Querying with Filters

Use SQLAlchemy filters to refine the query:

In [7]:
session.query(Widget).filter(Widget.timestamp.between("2016-11-01", "2016-11-30")).frame()

,id,name,kind,units
timestamp,,,,
2016-11-02 06:22:00,64,ponds,bopper,52
2016-11-04 15:16:00,65,maintenance,fizzer,83
2016-11-13 10:13:00,66,binder,fizzer,67
2016-11-17 14:20:00,67,boosts,buzzer,12
2016-11-22 14:27:00,68,address,buzzer,69
2016-11-29 21:49:00,69,integrity,bopper,54


Alternatively, use the `within()` method to query an instrumented attribute with a pandas index:

In [8]:
index = pandas.period_range("2016-11-01", "2016-11-30", freq="W")
session.query(Widget).filter(Widget.timestamp.within(index)).frame()

,id,name,kind,units
timestamp,,,,
2016-11-02 06:22:00,64,ponds,bopper,52
2016-11-04 15:16:00,65,maintenance,fizzer,83
2016-11-13 10:13:00,66,binder,fizzer,67
2016-11-17 14:20:00,67,boosts,buzzer,12
2016-11-22 14:27:00,68,address,buzzer,69
2016-11-29 21:49:00,69,integrity,bopper,54


### Overriding Defaults

Pass arguments to `pandas.read_sql()` into the `frame()` method:

In [9]:
session.query(Widget).frame(index_col="id").head()

,timestamp,name,kind,units
id,,,,
1,2016-01-05 10:07:00,injuries,bopper,54
2,2016-01-08 08:23:00,trick,buzzer,55
3,2016-01-12 07:54:00,flashlights,bopper,93
4,2016-01-24 13:10:00,purposes,fizzer,48
5,2016-01-29 12:13:00,subsystems,bopper,34


## Writing DataFrame to Session

Use the `add_dataframe()` method of the `RedPanda` session to place `DataFrame` rows into the session as models:

In [10]:
from datetime import datetime
dfdata = {
    "kind": "newkind", 
    "name": "newname", 
    "units": 1000, 
    "timestamp": datetime.utcnow()}
addme = pandas.DataFrame(dfdata, index=[0])
addme

,kind,name,timestamp,units
0,newkind,newname,2016-08-13 15:12:28.516306,1000


In [11]:
session.add_dataframe(Widget, addme)
session.commit()
session.query(Widget).filter(Widget.kind=="newkind").frame()

,id,name,kind,units
timestamp,,,,
2016-08-13 15:12:28.516306,76,newname,newkind,1000
